<h1>PROYECTO FINAL</h1>

![badge](author-badge.svg)

<div>
    <img src="vaccine.jpg"/>
</div>

<h2>Flu Shot Learning: Predict H1N1 and Seasonal Flu Vaccines</h2>

https://www.drivendata.org/competitions/66/flu-shot-learning/page/210/

¿Puedes predecir si las personas se pusieron las vacunas contra la H1N1 y la gripe estacional usando la información que compartieron sobre sus antecedentes, opiniones y comportamientos de salud?

En este desafío, veremos la vacunación, una medida clave de salud pública para combatir enfermedades infecciosas. Las vacunas proporcionan inmunización a los individuos, y suficiente inmunización en una comunidad puede reducir aún más la propagación de enfermedades a través de la "inmunidad de grupo."

Al momento del lanzamiento de esta competencia, las vacunas para el virus del COVID-19 todavía están en desarrollo y no están disponibles. En cambio, la competencia revisará la respuesta de salud pública a una pandemia respiratoria importante y reciente. A partir de la primavera de 2009, una pandemia causada por el virus de la influenza H1N1, conocido coloquialmente como "gripe porcina," se extendió por todo el mundo. Los investigadores estiman que en el primer año fue responsable de entre 151,000 y 575,000 muertes a nivel global.

# Importación de las librerías

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

import joblib
import pickle
import datetime

import os

from joblib import dump, load

from sklearn.preprocessing import LabelEncoder

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score, roc_auc_score, auc, roc_curve, confusion_matrix, ConfusionMatrixDisplay

from sklearn.tree import DecisionTreeClassifier

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from keras.regularizers import l2

import tkinter as tk
from tkinter import ttk
from tkinter import messagebox

import sweetviz as sv

from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import ADASYN
from imblearn.over_sampling import SMOTE

from xgboost import XGBClassifier

from sklearn.impute import KNNImputer

from lazypredict import LazyClassifier

import warnings

import math

from datetime import datetime



In [ ]:
start_time = datetime.now()

In [ ]:
warnings.filterwarnings("ignore")

# Creamos el PDF

In [ ]:
# pdf_path = "results.pdf"
# c = canvas.Canvas(pdf_path, pagesize=letter)
# width, height = letter

# Interfaz

In [ ]:
def update_values():
    global use_moda_train, delete_null_values_train, use_knn_imputer_train
    global use_moda_test, use_knn_imputer_test
    global num_neighbors_knn, number_of_best_features
    global use_logistic_regression_h1n1_vaccine, use_random_forest_h1n1_vaccine, use_ada_boost_classifier_h1n1_vaccine, use_xgb_classifier_h1n1_vaccine, use_red_neuronal_h1n1_vaccine
    global use_logistic_regression_seasonal_vaccine, use_random_forest_seasonal_vaccine, use_ada_boost_classifier_seasonal_vaccine, use_xgb_classifier_seasonal_vaccine, use_red_neuronal_seasonal_vaccine
    global epochs_red_neuronal, use_grid_search, use_random_search
    global use_standard_scaler, use_minmax_scaler, not_use_scaler
    global use_smote, use_adasyn, use_nearmiss, not_use_balanced

    use_moda_train = bool_var_use_moda_train.get()
    delete_null_values_train = bool_var_delete_null_values_train.get()
    use_knn_imputer_train = bool_var_use_knn_imputer_train.get()
    use_moda_test = bool_var_use_moda_test.get()
    use_knn_imputer_test = bool_var_use_knn_imputer_test.get()
    
    try:
        num_neighbors_knn = int(entry_num_neighbors_knn.get())
        if num_neighbors_knn < 1 or num_neighbors_knn > 20:
            raise ValueError
    except ValueError:
        messagebox.showerror("Error", "El número de vecinos debe estar entre 1 y 20")
        return

    try:
        number_of_best_features = int(entry_number_of_best_features.get())
        if number_of_best_features < 1 or number_of_best_features > 20:
            raise ValueError
    except ValueError:
        messagebox.showerror("Error", "El número de mejores características debe estar entre 1 y 20")
        return

    use_logistic_regression_h1n1_vaccine = bool_var_use_logistic_regression_h1n1_vaccine.get()
    use_random_forest_h1n1_vaccine = bool_var_use_random_forest_h1n1_vaccine.get()
    use_ada_boost_classifier_h1n1_vaccine = bool_var_use_ada_boost_classifier_h1n1_vaccine.get()
    use_xgb_classifier_h1n1_vaccine = bool_var_use_xgb_classifier_h1n1_vaccine.get()
    use_red_neuronal_h1n1_vaccine = bool_var_use_red_neuronal_h1n1_vaccine.get()

    use_logistic_regression_seasonal_vaccine = bool_var_use_logistic_regression_seasonal_vaccine.get()
    use_random_forest_seasonal_vaccine = bool_var_use_random_forest_seasonal_vaccine.get()
    use_ada_boost_classifier_seasonal_vaccine = bool_var_use_ada_boost_classifier_seasonal_vaccine.get()
    use_xgb_classifier_seasonal_vaccine = bool_var_use_xgb_classifier_seasonal_vaccine.get()
    use_red_neuronal_seasonal_vaccine = bool_var_use_red_neuronal_seasonal_vaccine.get()
    
    try:
        epochs_red_neuronal = int(entry_epochs_red_neuronal.get())
    except ValueError:
        messagebox.showerror("Error", "El número de épocas debe ser un número entero")
        return

    use_grid_search = bool_var_use_grid_search.get()
    use_random_search = bool_var_use_random_search.get()
    use_standard_scaler = bool_var_use_standard_scaler.get()
    use_minmax_scaler = bool_var_use_minmax_scaler.get()
    not_use_scaler = bool_var_not_use_scaler.get()
    use_smote = bool_var_use_smote.get()
    use_adasyn = bool_var_use_adasyn.get()
    use_nearmiss = bool_var_use_nearmiss.get()
    not_use_balanced = bool_var_not_use_balanced.get()
    
    print("use_moda_train:", use_moda_train)
    print("delete_null_values_train:", delete_null_values_train)
    print("use_knn_imputer_train:", use_knn_imputer_train)
    print("use_moda_test:", use_moda_test)
    print("use_knn_imputer_test:", use_knn_imputer_test)
    print("num_neighbors_knn:", num_neighbors_knn)
    print("number_of_best_features:", number_of_best_features)
    print("use_logistic_regression_h1n1_vaccine:", use_logistic_regression_h1n1_vaccine)
    print("use_random_forest_h1n1_vaccine:", use_random_forest_h1n1_vaccine)
    print("use_ada_boost_classifier_h1n1_vaccine:", use_ada_boost_classifier_h1n1_vaccine)
    print("use_xgb_classifier_h1n1_vaccine:", use_xgb_classifier_h1n1_vaccine)
    print("use_red_neuronal_h1n1_vaccine:", use_red_neuronal_h1n1_vaccine)
    print("use_logistic_regression_seasonal_vaccine:", use_logistic_regression_seasonal_vaccine)
    print("use_random_forest_seasonal_vaccine:", use_random_forest_seasonal_vaccine)
    print("use_ada_boost_classifier_seasonal_vaccine:", use_ada_boost_classifier_seasonal_vaccine)
    print("use_xgb_classifier_seasonal_vaccine:", use_xgb_classifier_seasonal_vaccine)
    print("use_red_neuronal_seasonal_vaccine:", use_red_neuronal_seasonal_vaccine)
    print("epochs_red_neuronal:", epochs_red_neuronal)
    print("use_grid_search:", use_grid_search)
    print("use_random_search:", use_random_search)
    print("use_standard_scaler:", use_standard_scaler)
    print("use_minmax_scaler:", use_minmax_scaler)
    print("not_use_scaler:", not_use_scaler)
    print("use_smote:", use_smote)
    print("use_adasyn:", use_adasyn)
    print("use_nearmiss:", use_nearmiss)
    print("not_use_balanced:", not_use_balanced)
    
    root.destroy()

# Initialize main window
root = tk.Tk()
root.title("Configuración de Variables")

# Variables
bool_var_use_moda_train = tk.BooleanVar(value=False)
bool_var_delete_null_values_train = tk.BooleanVar(value=False)
bool_var_use_knn_imputer_train = tk.BooleanVar(value=True)
bool_var_use_moda_test = tk.BooleanVar(value=False)
bool_var_use_knn_imputer_test = tk.BooleanVar(value=True)
bool_var_use_logistic_regression_h1n1_vaccine = tk.BooleanVar(value=True)
bool_var_use_random_forest_h1n1_vaccine = tk.BooleanVar(value=False)
bool_var_use_ada_boost_classifier_h1n1_vaccine = tk.BooleanVar(value=False)
bool_var_use_xgb_classifier_h1n1_vaccine = tk.BooleanVar(value=False)
bool_var_use_red_neuronal_h1n1_vaccine = tk.BooleanVar(value=False)
bool_var_use_logistic_regression_seasonal_vaccine = tk.BooleanVar(value=True)
bool_var_use_random_forest_seasonal_vaccine = tk.BooleanVar(value=False)
bool_var_use_ada_boost_classifier_seasonal_vaccine = tk.BooleanVar(value=False)
bool_var_use_xgb_classifier_seasonal_vaccine = tk.BooleanVar(value=False)
bool_var_use_red_neuronal_seasonal_vaccine = tk.BooleanVar(value=False)
bool_var_use_grid_search = tk.BooleanVar(value=True)
bool_var_use_random_search = tk.BooleanVar(value=False)
bool_var_use_standard_scaler = tk.BooleanVar(value=False)
bool_var_use_minmax_scaler = tk.BooleanVar(value=False)
bool_var_not_use_scaler = tk.BooleanVar(value=True)
bool_var_use_smote = tk.BooleanVar(value=False)
bool_var_use_adasyn = tk.BooleanVar(value=False)
bool_var_use_nearmiss = tk.BooleanVar(value=False)
bool_var_not_use_balanced = tk.BooleanVar(value=True)

# Functions to ensure only one selection per group
def disable_other_null_value_methods_train():
    bool_var_delete_null_values_train.set(False)
    bool_var_use_knn_imputer_train.set(False)

def disable_other_null_value_methods_knn_train():
    bool_var_use_moda_train.set(False)
    bool_var_delete_null_values_train.set(False)

def disable_other_null_value_methods_delete_train():
    bool_var_use_moda_train.set(False)
    bool_var_use_knn_imputer_train.set(False)

def disable_other_null_value_methods_test():
    bool_var_use_knn_imputer_test.set(False)

def disable_other_null_value_methods_knn_test():
    bool_var_use_moda_test.set(False)

def disable_other_models_h1n1():
    bool_var_use_random_forest_h1n1_vaccine.set(False)
    bool_var_use_red_neuronal_h1n1_vaccine.set(False)
    bool_var_use_ada_boost_classifier_h1n1_vaccine.set(False)
    bool_var_use_xgb_classifier_h1n1_vaccine.set(False)

def disable_other_models_rf_h1n1():
    bool_var_use_logistic_regression_h1n1_vaccine.set(False)
    bool_var_use_red_neuronal_h1n1_vaccine.set(False)
    bool_var_use_ada_boost_classifier_h1n1_vaccine.set(False)
    bool_var_use_xgb_classifier_h1n1_vaccine.set(False)

def disable_other_models_ada_h1n1():
    bool_var_use_logistic_regression_h1n1_vaccine.set(False)
    bool_var_use_random_forest_h1n1_vaccine.set(False)
    bool_var_use_red_neuronal_h1n1_vaccine.set(False)
    bool_var_use_xgb_classifier_h1n1_vaccine.set(False)

def disable_other_models_nn_h1n1():
    bool_var_use_logistic_regression_h1n1_vaccine.set(False)
    bool_var_use_random_forest_h1n1_vaccine.set(False)
    bool_var_use_ada_boost_classifier_h1n1_vaccine.set(False)
    bool_var_use_xgb_classifier_h1n1_vaccine.set(False)

def disable_other_models_xgb_h1n1():
    bool_var_use_logistic_regression_h1n1_vaccine.set(False)
    bool_var_use_random_forest_h1n1_vaccine.set(False)
    bool_var_use_red_neuronal_h1n1_vaccine.set(False)
    bool_var_use_ada_boost_classifier_h1n1_vaccine.set(False)

def disable_other_models_seasonal():
    bool_var_use_random_forest_seasonal_vaccine.set(False)
    bool_var_use_red_neuronal_seasonal_vaccine.set(False)
    bool_var_use_ada_boost_classifier_seasonal_vaccine.set(False)
    bool_var_use_xgb_classifier_seasonal_vaccine.set(False)

def disable_other_models_rf_seasonal():
    bool_var_use_logistic_regression_seasonal_vaccine.set(False)
    bool_var_use_red_neuronal_seasonal_vaccine.set(False)
    bool_var_use_ada_boost_classifier_seasonal_vaccine.set(False)
    bool_var_use_xgb_classifier_seasonal_vaccine.set(False)

def disable_other_models_ada_seasonal():
    bool_var_use_logistic_regression_seasonal_vaccine.set(False)
    bool_var_use_random_forest_seasonal_vaccine.set(False)
    bool_var_use_red_neuronal_seasonal_vaccine.set(False)
    bool_var_use_xgb_classifier_seasonal_vaccine.set(False)

def disable_other_models_nn_seasonal():
    bool_var_use_logistic_regression_seasonal_vaccine.set(False)
    bool_var_use_random_forest_seasonal_vaccine.set(False)
    bool_var_use_ada_boost_classifier_seasonal_vaccine.set(False)
    bool_var_use_xgb_classifier_seasonal_vaccine.set(False)

def disable_other_models_xgb_seasonal():
    bool_var_use_logistic_regression_seasonal_vaccine.set(False)
    bool_var_use_random_forest_seasonal_vaccine.set(False)
    bool_var_use_red_neuronal_seasonal_vaccine.set(False)
    bool_var_use_ada_boost_classifier_seasonal_vaccine.set(False)

def disable_other_search_methods():
    bool_var_use_random_search.set(False)

def disable_other_search_methods_random():
    bool_var_use_grid_search.set(False)

def disable_other_scalers():
    bool_var_use_standard_scaler.set(False)
    bool_var_not_use_scaler.set(False)

def disable_other_scalers_standard():
    bool_var_use_minmax_scaler.set(False)
    bool_var_not_use_scaler.set(False)

def disable_other_scalers_not():
    bool_var_use_minmax_scaler.set(False)
    bool_var_use_standard_scaler.set(False)

def disable_other_balancing_methods():
    bool_var_use_adasyn.set(False)
    bool_var_use_nearmiss.set(False)
    bool_var_not_use_balanced.set(False)

def disable_other_balancing_methods_adasyn():
    bool_var_use_smote.set(False)
    bool_var_use_nearmiss.set(False)
    bool_var_not_use_balanced.set(False)

def disable_other_balancing_methods_nearmiss():
    bool_var_use_smote.set(False)
    bool_var_use_adasyn.set(False)
    bool_var_not_use_balanced.set(False)

def disable_other_balancing_methods_not():
    bool_var_use_smote.set(False)
    bool_var_use_adasyn.set(False)
    bool_var_use_nearmiss.set(False)

# Layout with added padding
pad_options = {'padx': 5, 'pady': 5}

ttk.Label(root, text="¿Qué proceso seguir para rellenar los valores nulos de los datos de entrenamiento?").grid(column=0, row=0, sticky=tk.W, **pad_options)
ttk.Checkbutton(root, text="Usar Moda", variable=bool_var_use_moda_train, command=disable_other_null_value_methods_train).grid(column=1, row=0, sticky=tk.W, **pad_options)
ttk.Checkbutton(root, text="Eliminar valores nulos", variable=bool_var_delete_null_values_train, command=disable_other_null_value_methods_delete_train).grid(column=2, row=0, sticky=tk.W, **pad_options)
ttk.Checkbutton(root, text="Usar KNNImputer", variable=bool_var_use_knn_imputer_train, command=disable_other_null_value_methods_knn_train).grid(column=3, row=0, sticky=tk.W, **pad_options)

ttk.Label(root, text="¿Qué proceso seguir para rellenar los valores nulos de los datos de test?").grid(column=0, row=1, sticky=tk.W, **pad_options)
ttk.Checkbutton(root, text="Usar Moda", variable=bool_var_use_moda_test, command=disable_other_null_value_methods_test).grid(column=1, row=1, sticky=tk.W, **pad_options)
ttk.Checkbutton(root, text="Usar KNNImputer", variable=bool_var_use_knn_imputer_test, command=disable_other_null_value_methods_knn_test).grid(column=2, row=1, sticky=tk.W, **pad_options)

ttk.Label(root, text="Número de vecinos para KNNImputer (1-20)").grid(column=0, row=2, sticky=tk.W, **pad_options)
entry_num_neighbors_knn = ttk.Entry(root)
entry_num_neighbors_knn.insert(0, "5")
entry_num_neighbors_knn.grid(column=1, row=2, sticky=tk.W, **pad_options)

ttk.Label(root, text="¿Con cuántas variables te quedas para el entrenamiento? (1-20)").grid(column=0, row=3, sticky=tk.W, **pad_options)
entry_number_of_best_features = ttk.Entry(root)
entry_number_of_best_features.insert(0, "10")
entry_number_of_best_features.grid(column=1, row=3, sticky=tk.W, **pad_options)

ttk.Label(root, text="¿Qué modelo eliges para entrenar h1n1_vaccine?").grid(column=0, row=4, sticky=tk.W, **pad_options)
ttk.Checkbutton(root, text="LogisticRegression", variable=bool_var_use_logistic_regression_h1n1_vaccine, command=disable_other_models_h1n1).grid(column=1, row=4, sticky=tk.W, **pad_options)
ttk.Checkbutton(root, text="RandomForestClassifier", variable=bool_var_use_random_forest_h1n1_vaccine, command=disable_other_models_rf_h1n1).grid(column=2, row=4, sticky=tk.W, **pad_options)
ttk.Checkbutton(root, text="AdaBoostClassifier", variable=bool_var_use_ada_boost_classifier_h1n1_vaccine, command=disable_other_models_ada_h1n1).grid(column=3, row=4, sticky=tk.W, **pad_options)
ttk.Checkbutton(root, text="XGBClassifier", variable=bool_var_use_xgb_classifier_h1n1_vaccine, command=disable_other_models_xgb_h1n1).grid(column=4, row=4, sticky=tk.W, **pad_options)
ttk.Checkbutton(root, text="Red Neuronal", variable=bool_var_use_red_neuronal_h1n1_vaccine, command=disable_other_models_nn_h1n1).grid(column=5, row=4, sticky=tk.W, **pad_options)

ttk.Label(root, text="¿Qué modelo eliges para entrenar seasonal_vaccine?").grid(column=0, row=5, sticky=tk.W, **pad_options)
ttk.Checkbutton(root, text="LogisticRegression", variable=bool_var_use_logistic_regression_seasonal_vaccine, command=disable_other_models_seasonal).grid(column=1, row=5, sticky=tk.W, **pad_options)
ttk.Checkbutton(root, text="RandomForestClassifier", variable=bool_var_use_random_forest_seasonal_vaccine, command=disable_other_models_rf_seasonal).grid(column=2, row=5, sticky=tk.W, **pad_options)
ttk.Checkbutton(root, text="AdaBoostClassifier", variable=bool_var_use_ada_boost_classifier_seasonal_vaccine, command=disable_other_models_ada_seasonal).grid(column=3, row=5, sticky=tk.W, **pad_options)
ttk.Checkbutton(root, text="XGBClassifier", variable=bool_var_use_xgb_classifier_seasonal_vaccine, command=disable_other_models_xgb_seasonal).grid(column=4, row=5, sticky=tk.W, **pad_options)
ttk.Checkbutton(root, text="Red Neuronal", variable=bool_var_use_red_neuronal_seasonal_vaccine, command=disable_other_models_nn_seasonal).grid(column=5, row=5, sticky=tk.W, **pad_options)

ttk.Label(root, text="Si se ha seleccionado red neuronal, ¿cuántas épocas?").grid(column=0, row=6, sticky=tk.W, **pad_options)
entry_epochs_red_neuronal = ttk.Entry(root)
entry_epochs_red_neuronal.insert(0, "20")
entry_epochs_red_neuronal.grid(column=1, row=6, sticky=tk.W, **pad_options)

ttk.Label(root, text="¿Qué tipo de búsqueda de hiperpárametros? (si no ha elegido red neuronal)").grid(column=0, row=7, sticky=tk.W, **pad_options)
ttk.Checkbutton(root, text="Grid Search", variable=bool_var_use_grid_search, command=disable_other_search_methods).grid(column=1, row=7, sticky=tk.W, **pad_options)
ttk.Checkbutton(root, text="Random Search", variable=bool_var_use_random_search, command=disable_other_search_methods_random).grid(column=2, row=7, sticky=tk.W, **pad_options)

ttk.Label(root, text="¿Qué tipo de escalamiento de datos?").grid(column=0, row=8, sticky=tk.W, **pad_options)
ttk.Checkbutton(root, text="Standard Scaler", variable=bool_var_use_standard_scaler, command=disable_other_scalers_standard).grid(column=1, row=8, sticky=tk.W, **pad_options)
ttk.Checkbutton(root, text="MinMax Scaler", variable=bool_var_use_minmax_scaler, command=disable_other_scalers).grid(column=2, row=8, sticky=tk.W, **pad_options)
ttk.Checkbutton(root, text="No usar escalamiento", variable=bool_var_not_use_scaler, command=disable_other_scalers_not).grid(column=3, row=8, sticky=tk.W, **pad_options)

ttk.Label(root, text="¿Qué tipo de balanceo de datos para clases minoritarias?").grid(column=0, row=9, sticky=tk.W, **pad_options)
ttk.Checkbutton(root, text="SMOTE", variable=bool_var_use_smote, command=disable_other_balancing_methods).grid(column=1, row=9, sticky=tk.W, **pad_options)
ttk.Checkbutton(root, text="ADASYN", variable=bool_var_use_adasyn, command=disable_other_balancing_methods_adasyn).grid(column=2, row=9, sticky=tk.W, **pad_options)
ttk.Checkbutton(root, text="NearMiss", variable=bool_var_use_nearmiss, command=disable_other_balancing_methods_nearmiss).grid(column=3, row=9, sticky=tk.W, **pad_options)
ttk.Checkbutton(root, text="No usar balanceo", variable=bool_var_not_use_balanced, command=disable_other_balancing_methods_not).grid(column=4, row=9, sticky=tk.W, **pad_options)

ttk.Button(root, text="Entrenar", command=update_values).grid(column=0, row=10, sticky=tk.W, **pad_options)

root.mainloop()


# Cargamos datasets

Cargamos el dataset de entrenamiento con las características

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/aaronmed/iabigdata_proyecto_final/main/dataset/training_set_features.csv')

df.head(5)

Cargamos el dataset de entrenamiento con las variables objetivos

In [ ]:
df_training_labels = pd.read_csv('https://raw.githubusercontent.com/aaronmed/iabigdata_proyecto_final/main/dataset/training_set_labels.csv')

Mezclamos ambos dataset por la columna identificadora (el id del encuestado)

In [ ]:
merged_df = pd.merge(df, df_training_labels, on='respondent_id', how='inner')

Cargamos el dataset de test

In [ ]:
test = pd.read_csv('https://raw.githubusercontent.com/aaronmed/iabigdata_proyecto_final/main/dataset/test_set_features.csv')

# Análisis de datos

Analizamos los valores únicos que existen con el objetivo de ver si son características categóricas o numéricas

In [ ]:
valores_unicos = merged_df.select_dtypes(include=['object']).apply(pd.Series.nunique)

print(valores_unicos)

Mostramos cuáles son los valores de las columnas de tipo objeto

In [ ]:
for columna in merged_df.select_dtypes(include=['object']):
    print(columna, ":", df[columna].unique())

In [ ]:
merged_df.describe()

In [ ]:
merged_df.info()

In [ ]:
palette_h1n1_vaccine = ['#D18C7F', '#85CAA0']
palette_seasonal_vaccine = ['#e74c3c', '#2ecc71']

In [ ]:
for col in merged_df.select_dtypes(include=['object']).columns:
    # Obtenemos las frecuencias de los valores
    value_counts = merged_df[col].value_counts()
    
    # Creamos el gráfico de barras
    plt.figure(figsize=(10, 6))
    value_counts.plot(kind='bar')
    plt.title(f'Frecuencia de valores en {col}')
    plt.xlabel('Valores')
    plt.ylabel('Frecuencia')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    
    # Mostramos el gráfico
    plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.countplot(x='age_group', hue='h1n1_vaccine', data=merged_df, palette=palette_h1n1_vaccine)
plt.title('Distribución de age_group vs. h1n1_vaccine')
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.countplot(x='age_group', hue='seasonal_vaccine', data=merged_df, palette=palette_seasonal_vaccine)
plt.title('Distribución de age_group vs. seasonal_vaccine')
plt.show()

In [ ]:
plt.figure(figsize=(8, 8))
merged_df['race'].value_counts().plot.pie(autopct='%1.1f%%')
plt.title('Proporción de categorías en race')
plt.ylabel('')
plt.show()

In [ ]:
colors_no_vaccine = ['#ff9999','#66b3ff','#99ff99','#ffcc99']
colors_vaccine = ['#c2c2f0','#ffb3e6','#c4e17f','#ff6666']

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(14, 7))

merged_df[merged_df['h1n1_vaccine'] == 0]['race'].value_counts().plot.pie(ax=ax[0], autopct='%1.1f%%', startangle=90, colors=colors_no_vaccine)
ax[0].set_title('Proporción de race (No h1n1_vaccine)')
ax[0].set_ylabel('')

merged_df[merged_df['h1n1_vaccine'] == 1]['race'].value_counts().plot.pie(ax=ax[1], autopct='%1.1f%%', startangle=90, colors=colors_vaccine)
ax[1].set_title('Proporción de race (Sí h1n1_vaccine)')
ax[1].set_ylabel('')

plt.show()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(14, 7))

merged_df[merged_df['seasonal_vaccine'] == 0]['race'].value_counts().plot.pie(ax=ax[0], autopct='%1.1f%%', startangle=90, colors=colors_no_vaccine)
ax[0].set_title('Proporción de race (No seasonal_vaccine)')
ax[0].set_ylabel('')

merged_df[merged_df['seasonal_vaccine'] == 1]['race'].value_counts().plot.pie(ax=ax[1], autopct='%1.1f%%', startangle=90, colors=colors_vaccine)
ax[1].set_title('Proporción de race (Sí seasonal_vaccine)')
ax[1].set_ylabel('')

plt.show()

In [ ]:
binary_columns = [col for col in merged_df.columns if merged_df[col].dropna().unique().tolist() in ([0, 1], [1, 0])]

In [ ]:
# Número de columnas por fila
cols_per_row = 3

# Número total de gráficas
total_plots = len(binary_columns)

# Número de filas necesarias
num_figures = math.ceil(total_plots / cols_per_row)

In [ ]:
# Generar las figuras necesarias
for fig_num in range(num_figures):
    # Crear una nueva figura
    plt.figure(figsize=(15, 5))
    
    # Determinar el rango de columnas para esta figura
    start_idx = fig_num * cols_per_row
    end_idx = min(start_idx + cols_per_row, total_plots)
    
    for i, col in enumerate(binary_columns[start_idx:end_idx]):
        plt.subplot(1, cols_per_row, i+1)
        sns.countplot(x=col, hue='h1n1_vaccine', data=merged_df, palette=palette_h1n1_vaccine)
        plt.title(f'{col} vs. h1n1_vaccine')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Generar las figuras necesarias
for fig_num in range(num_figures):
    # Crear una nueva figura
    plt.figure(figsize=(15, 5))
    
    # Determinar el rango de columnas para esta figura
    start_idx = fig_num * cols_per_row
    end_idx = min(start_idx + cols_per_row, total_plots)
    
    for i, col in enumerate(binary_columns[start_idx:end_idx]):
        plt.subplot(1, cols_per_row, i+1)
        sns.countplot(x=col, hue='seasonal_vaccine', data=merged_df, palette=palette_seasonal_vaccine)
        plt.title(f'{col} vs. seasonal_vaccine')
    
    plt.tight_layout()
    plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(x='income_poverty', hue='h1n1_vaccine', data=merged_df, palette=palette_h1n1_vaccine)
plt.title('Income Poverty vs. H1N1 Vaccine')
plt.xlabel('Income Poverty')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.legend(title='H1N1 Vaccine', loc='upper right')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(x='income_poverty', hue='seasonal_vaccine', data=merged_df, palette=palette_seasonal_vaccine)
plt.title('Income Poverty vs. Seasonal Vaccine')
plt.xlabel('Income Poverty')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.legend(title='Seasonal Vaccine', loc='upper right')
plt.tight_layout()
plt.show()

In [ ]:
view_sweetviz_h1n1_vaccine = False
view_sweetviz_seasonal_vaccine = False

In [ ]:
if view_sweetviz_h1n1_vaccine:
    report_h1n1_vaccine = sv.analyze(merged_df, target_feat='h1n1_vaccine')

    report_h1n1_vaccine.show_notebook(layout="widescreen")

In [ ]:
if view_sweetviz_seasonal_vaccine:
    report_seasonal_vaccine = sv.analyze(merged_df, target_feat='seasonal_vaccine')

    report_seasonal_vaccine.show_notebook(layout="widescreen")

In [ ]:
# Función para etiquetar los valores de los gráficos de tarta
def label_function(val):
    return 'Vacunado' if val == 1 else 'No vacunado'

# Contamos la cantidad de vacunados (valor 1) y no vacunados (valor 0) para cada vacuna
h1n1_counts = merged_df['h1n1_vaccine'].value_counts()
seasonal_counts = merged_df['seasonal_vaccine'].value_counts()

# Creamos el primer gráfico de tarta para h1n1_vaccine
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.pie(h1n1_counts, labels=h1n1_counts.index.map(label_function), autopct='%1.1f%%', startangle=140, colors=palette_h1n1_vaccine)
plt.title('Proporción vacunados de la gripe H1N1')
plt.axis('equal')

# Creamos el segundo gráfico de tarta para seasonal_vaccine
plt.subplot(1, 2, 2)
plt.pie(seasonal_counts, labels=seasonal_counts.index.map(label_function), autopct='%1.1f%%', startangle=140, colors=palette_seasonal_vaccine)
plt.title('Proporción de vacunados de la gripe estacional')
plt.axis('equal')

plt.show()

# Conversión de datos

In [ ]:
df_for_label_encoder = pd.concat([merged_df, test])

categorical_columns = df_for_label_encoder.select_dtypes(include=['object'])

encoders = {}

for col in categorical_columns.columns:
    le = LabelEncoder()
    le.fit_transform(df_for_label_encoder[col])
    encoders[col] = le 
    
for col, le in encoders.items():
    merged_df[col] = le.transform(merged_df[col])

In [ ]:
for col in categorical_columns.columns:
    print(col)
    
for col in categorical_columns.columns:
    print(encoders[col].classes_)

# Matriz de correlación

In [ ]:
correlation_matrix = merged_df.corr()

# Visualizar la matriz de correlación
plt.figure(figsize=(25, 20))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Matriz de Correlación')
plt.show()

In [ ]:
correlation_matrix = merged_df.corr()

limit_filtered_correlation_matrix = 0.20

filtered_corr_matrix = correlation_matrix[(np.abs(correlation_matrix) > limit_filtered_correlation_matrix) & (correlation_matrix != 1.0)]

plt.figure(figsize=(25, 20))
sns.heatmap(filtered_corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title(f'Matriz de correlación (Valores absolutos > {limit_filtered_correlation_matrix})')
plt.show()

# Gráficas de visualización para análisis de datos

In [ ]:
combinations = merged_df.groupby(['h1n1_vaccine', 'doctor_recc_h1n1']).size().reset_index(name='count')

# Crear una columna combinada para usar como etiquetas en el eje x
combinations['combination'] = combinations.apply(
    lambda row: f"V: {'Sí' if row['h1n1_vaccine'] == 1 else 'No'} y R: {'Sí' if row['doctor_recc_h1n1'] == 1 else 'No'}", 
    axis=1
)

# Crear un gráfico de barras con tamaño ajustado
fig, ax = plt.subplots(figsize=(10, 6))
combinations.plot(kind='bar', x='combination', y='count', ax=ax, legend=False, color='salmon')

# Etiquetas y título del gráfico
ax.set_xlabel('Combinaciones de Vacunación para el H1N1 y Recomendación')
ax.set_ylabel('Número de Encuestados')
ax.set_title('Número de Encuestados por Combinaciones de h1n1_vaccine y doctor_recc_h1n1')

# Colocar los labels en diagonal con ajuste de alineación y tamaño de fuente
plt.xticks(rotation=0, ha='center', fontsize=10)

# Ajustar el diseño para que las etiquetas no se corten
plt.tight_layout()

plt.show()

In [ ]:
combinations = merged_df.groupby(['seasonal_vaccine', 'doctor_recc_seasonal']).size().reset_index(name='count')

# Crear una columna combinada para usar como etiquetas en el eje x
combinations['combination'] = combinations.apply(
    lambda row: f"V: {'Sí' if row['seasonal_vaccine'] == 1 else 'No'} y R: {'Sí' if row['doctor_recc_seasonal'] == 1 else 'No'}", 
    axis=1
)

# Crear un gráfico de barras con tamaño ajustado
fig, ax = plt.subplots(figsize=(10, 6))
combinations.plot(kind='bar', x='combination', y='count', ax=ax, legend=False, color='tan')

# Etiquetas y título del gráfico
ax.set_xlabel('Combinaciones de Vacunación para gripe estacional y Recomendación')
ax.set_ylabel('Número de Encuestados')
ax.set_title('Número de Encuestados por Combinaciones de seasonal_vaccine y doctor_recc_seasonal')

# Colocar los labels en diagonal con ajuste de alineación y tamaño de fuente
plt.xticks(rotation=0, ha='center', fontsize=10)

# Ajustar el diseño para que las etiquetas no se corten
plt.tight_layout()

plt.show()

# Tratamiento de valores nulos

In [ ]:
# Identificar columnas con valores nulos
columnas_con_nulos = merged_df.columns[merged_df.isnull().any()].tolist()

nulos_por_columna = merged_df[columnas_con_nulos].isnull().sum()

In [ ]:
# Crear la gráfica
plt.figure(figsize=(12, 8))
nulos_por_columna.plot(kind='bar', color='skyblue')
plt.title('Número de valores nulos por columna')
plt.xlabel('Columnas')
plt.ylabel('Número de valores nulos')
plt.xticks(rotation=45)
plt.grid(axis='y')

# Mostrar la gráfica
plt.show()

Rellenar las columnas con valores nulos con la moda de cada columna

In [ ]:
if use_moda_train:
    moda = test[columna].mode()[0]
    test[columna].fillna(moda, inplace=True)
    print(f'{columna} - {moda}')

Usamos KNNImputer para rellenar los valores nulos

In [ ]:
if use_knn_imputer_train:
    imputer = KNNImputer(n_neighbors=num_neighbors_knn)
    imputed_data = imputer.fit_transform(merged_df)
    imputed_data = np.round(imputed_data).astype(int)
    merged_df = pd.DataFrame(imputed_data, columns=merged_df.columns, index=merged_df.index)

Eliminamos los valores nulos

In [ ]:
if delete_null_values_train:
    rows_before = len(merged_df)  # Número de filas antes de eliminar valores faltantes

    merged_df = merged_df.dropna()  # Eliminar filas con valores faltantes

    rows_after = len(merged_df)  # Número de filas después de eliminar valores faltantes

    print("Número de filas antes de eliminar valores faltantes:", rows_before)
    print("Número de filas después de eliminar valores faltantes:", rows_after)
    print("Número de filas eliminadas:", rows_before - rows_after)

# Selección de características más importantes

Eliminamos la clase identificador

In [ ]:
merged_df = merged_df.drop('respondent_id', axis=1)

In [ ]:
X = merged_df.drop(columns=['h1n1_vaccine', 'seasonal_vaccine'])

y_h1n1_vaccine = merged_df['h1n1_vaccine']   # Primera columna objetivo (h1n1_vaccine)
y_seasonal_vaccine = merged_df['seasonal_vaccine']   # Segunda columna objetivo (seasonal_vaccine)

In [ ]:
# Crear el selector de características para la primera columna objetivo
selector_h1n1_vaccine = SelectKBest(score_func=f_classif, k=number_of_best_features)  # Aquí especificas el método de selección de características y el número de características que deseas seleccionar

# Aplicar el selector a tus datos para la primera columna objetivo
X_new_h1n1_vaccine = selector_h1n1_vaccine.fit_transform(X, y_h1n1_vaccine)

# Obtener los índices de las características seleccionadas para la primera columna objetivo
selected_features_h1n1_vaccine = selector_h1n1_vaccine.get_support(indices=True)

# Obtener los nombres de las características seleccionadas para la primera columna objetivo
selected_features_names_h1n1_vaccine = X.columns[selected_features_h1n1_vaccine]

# Imprimir los nombres de las características seleccionadas para la primera columna objetivo
print("Características seleccionadas para h1n1_vaccine:", selected_features_names_h1n1_vaccine)

# Obtener los pesos de cada característica para la primera columna objetivo
feature_scores_h1n1_vaccine = selector_h1n1_vaccine.scores_

# Crear un DataFrame con los nombres de las características y sus pesos para la primera columna objetivo
feature_scores_df_h1n1_vaccine = pd.DataFrame({'Feature': X.columns, 'Score': feature_scores_h1n1_vaccine})

# Ordenar el DataFrame por los pesos en orden descendente para la primera columna objetivo
feature_scores_df_h1n1_vaccine = feature_scores_df_h1n1_vaccine.sort_values(by='Score', ascending=False)

# Mostrar la gráfica de barras con los pesos de cada característica para la primera columna objetivo
plt.figure(figsize=(10, 6))
plt.barh(feature_scores_df_h1n1_vaccine['Feature'], feature_scores_df_h1n1_vaccine['Score'], color='skyblue')
plt.xlabel('Puntuación')
plt.ylabel('Característica')
plt.title('Puntuación de las características para H1N1_vaccine')
plt.show()

# Imprimir el DataFrame ordenado por los pesos para la primera columna objetivo
print("Puntuación de las características para H1N1_vaccine:")
print(feature_scores_df_h1n1_vaccine)

In [ ]:
# Crear el selector de características para la segunda columna objetivo
selector_seasonal_vaccine = SelectKBest(score_func=f_classif, k=number_of_best_features)  # Aquí especificas el método de selección de características y el número de características que deseas seleccionar

# Aplicar el selector a tus datos para la segunda columna objetivo
X_new_seasonal_vaccine = selector_seasonal_vaccine.fit_transform(X, y_seasonal_vaccine)

# Obtener los índices de las características seleccionadas para la segunda columna objetivo
selected_features_seasonal_vaccine = selector_seasonal_vaccine.get_support(indices=True)

# Obtener los nombres de las características seleccionadas para la segunda columna objetivo
selected_features_names_seasonal_vaccine = X.columns[selected_features_seasonal_vaccine]

# Imprimir los nombres de las características seleccionadas para la segunda columna objetivo
print("Características seleccionadas para seasonal_vaccine:", selected_features_names_seasonal_vaccine)

# Obtener los pesos de cada característica para la segunda columna objetivo
feature_scores_seasonal_vaccine = selector_seasonal_vaccine.scores_

# Crear un DataFrame con los nombres de las características y sus pesos para la segunda columna objetivo
feature_scores_df_seasonal_vaccine = pd.DataFrame({'Feature': X.columns, 'Score': feature_scores_seasonal_vaccine})

# Ordenar el DataFrame por los pesos en orden descendente para la segunda columna objetivo
feature_scores_df_seasonal_vaccine = feature_scores_df_seasonal_vaccine.sort_values(by='Score', ascending=False)

# Mostrar la gráfica de barras con los pesos de cada característica para la segunda columna objetivo
plt.figure(figsize=(10, 6))
plt.barh(feature_scores_df_seasonal_vaccine['Feature'], feature_scores_df_seasonal_vaccine['Score'], color='skyblue')
plt.xlabel('Puntuación')
plt.ylabel('Característica')
plt.title('Puntuación de las características para seasonal_vaccine:')
plt.show()

# Imprimir el DataFrame ordenado por los pesos para la segunda columna objetivo
print("Puntuación de las características para seasonal_vaccine:")
print(feature_scores_df_seasonal_vaccine)

# Definimos métodos para el entrenamiento del modelo

In [ ]:
def split_x_y(X, y, top_features):
    # Seleccionar las mejores características
    X_mejores_caracteristicas = X[top_features]
    
    # Dividir los datos en conjunto de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X_mejores_caracteristicas, y, test_size=0.2, random_state=42)
    
    if use_smote:
        print("Aplicando SMOTE...")
        smote = SMOTE(random_state=42)
        X_train, y_train = smote.fit_resample(X_train, y_train)
        
    if use_adasyn:
        print("Aplicando ADASYN...")
        adasyn = ADASYN(random_state=42)
        X_train, y_train = adasyn.fit_resample(X_train, y_train)
    
    if use_nearmiss:
        print("Aplicando NearMiss...")
        nm = NearMiss(version=2)
        X_train, y_train = nm.fit_resample(X_train, y_train)
    
    return X_train, X_test, y_train, y_test

In [ ]:
def apply_scaler(X):
    
    scaler = None
    
    if use_minmax_scaler:
        print("Aplicando MinMaxScaler...")
        scaler = MinMaxScaler()
        scaled_data = scaler.fit_transform(X)

        X = pd.DataFrame(scaled_data, columns=X.columns)
        
    if use_standard_scaler:
        print("Aplicando StandardScaler...")
        scaler = StandardScaler()
        scaled_data = scaler.fit_transform(X)

        X = pd.DataFrame(scaled_data, columns=X.columns)
        
    return X, scaler

In [ ]:
def entrenar_y_evaluar(modelo, search_cv, X_train, X_test, y_train, y_test):
    # Entrenar el modelo
    search_cv.fit(X_train, y_train)
    
    print('¡Entrenamiento finalizado!')

    # Obtener el mejor modelo entrenado
    mejor_modelo = search_cv.best_estimator_

    # Realizar predicciones
    probabilities = mejor_modelo.predict_proba(X_test)[:, 1]
    predictions = mejor_modelo.predict(X_test)

    accuracy_result = accuracy_score(y_test, mejor_modelo.predict(X_test))
    auc_result = roc_auc_score(y_test, probabilities, average='macro')

    print("Accuracy:", accuracy_result)
    print("AUC:", auc_result)
    
    print(f'Mejores parámetros encontrados: {search_cv.best_params_}')
    print("---------------------------------------------------------------------------")
    
    # Graficar la curva ROC
    fpr, tpr, _ = roc_curve(y_test, probabilities)
    roc_auc = auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f' Curva ROC - {modelo.__class__.__name__}')
    plt.legend(loc="lower right")
    plt.show()

    # Graficar la matriz de confusión
    cm = confusion_matrix(y_test, predictions)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(cmap=plt.cm.Blues)
    plt.title(f'Matriz de confusión - {modelo.__class__.__name__}')
    plt.show()
    
    return mejor_modelo

In [ ]:
def entrenar_y_evaluar_modelo_con_search(modelo, parametros, X, y, top_features, search_type):
    # Dividir los datos en conjunto de entrenamiento y prueba
    X_train, X_test, y_train, y_test = split_x_y(X, y, top_features)

    # Crear el objeto de búsqueda
    if search_type == 'grid':
        print(f"Empezando búsqueda de hiperpárametros para {modelo.__class__.__name__} con GridSearchCV")
        search_cv = GridSearchCV(estimator=modelo, param_grid=parametros, scoring='roc_auc', cv=5, n_jobs=-1)
    elif search_type == 'random':
        print(f"Empezando búsqueda de hiperpárametros para {modelo.__class__.__name__} con RandomSearchCV")
        search_cv = RandomizedSearchCV(estimator=modelo, param_distributions=parametros, scoring='roc_auc', cv=5, n_jobs=-1, n_iter=100)
    else:
        raise ValueError("search_type debe ser 'grid' o 'random'")
    
    return entrenar_y_evaluar(modelo, search_cv, X_train, X_test, y_train, y_test)

# Evaluamos con LazyPredict

In [ ]:
X, scaler = apply_scaler(X)

In [ ]:
def evaluate_lazy_classifier(X, y, top_features, target):
    print(f"Evaluando modelos con LazyClassifier para {target}...")
    
    lazy_classifier = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
    
    X_train, X_test, y_train, y_test = split_x_y(X, y, top_features)
    
    models, predictions = lazy_classifier.fit(X_train, X_test, y_train, y_test)
    
    return models

In [ ]:
use_lazy_predict = True

In [ ]:
if use_lazy_predict:
    predictions_h1n1_vaccine_lazypredict = evaluate_lazy_classifier(X, y_h1n1_vaccine, selected_features_names_h1n1_vaccine, 'h1n1_vaccine')

    predictions_h1n1_vaccine_lazypredict

In [ ]:
if use_lazy_predict:
    predictions_seasonal_vaccine_lazypredict = evaluate_lazy_classifier(X, y_seasonal_vaccine, selected_features_names_seasonal_vaccine, 'seasonal_vaccine')

    predictions_seasonal_vaccine_lazypredict

# Entrenamiento de modelo

In [ ]:
model_h1n1_vaccine = None
model_seasonal_vaccine = None

## LogisticRegression

In [ ]:
param_logistic_regression = {
    'penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'class_weight': ['balanced', 'balanced_subsample', None],
    'max_iter': [100, 500, 1000, 2000],
    'multi_class': ['auto', 'ovr', 'multinomial'],
    'fit_intercept': [True, False],
}

### h1n1_vaccine

In [ ]:

if use_grid_search and use_logistic_regression_h1n1_vaccine:
    print("Vamos a realizar búsqueda de hiperparámetros para h1n1_vaccine con GridSearchCV y el modelo LogisticRegression...")
    model_h1n1_vaccine = entrenar_y_evaluar_modelo_con_search(
        LogisticRegression(),
        param_logistic_regression,
        X,
        y_h1n1_vaccine,
        selected_features_names_h1n1_vaccine,
        'grid'
    )

In [ ]:
if use_random_search and use_logistic_regression_h1n1_vaccine:
    print("Vamos a realizar búsqueda de hiperparámetros para h1n1_vaccine con RandomSearchCV y el modelo LogisticRegression...")
    model_h1n1_vaccine = entrenar_y_evaluar_modelo_con_search(
        LogisticRegression(),
        param_logistic_regression,
        X,
        y_h1n1_vaccine,
        selected_features_names_h1n1_vaccine,
        'random'
    )

### seasonal_vaccine

In [ ]:
if use_grid_search and use_logistic_regression_seasonal_vaccine:
    print("Vamos a realizar búsqueda de hiperparámetros para seasonal_vaccine con GridSearchCV y el modelo LogisticRegression...")
    model_seasonal_vaccine = entrenar_y_evaluar_modelo_con_search(
        LogisticRegression(),
        param_logistic_regression,
        X,
        y_seasonal_vaccine,
        selected_features_names_seasonal_vaccine,
        'grid'
    )

In [ ]:
if use_random_search and use_logistic_regression_seasonal_vaccine:
    print("Vamos a realizar búsqueda de hiperparámetros para seasonal_vaccine con RandomSearchCV y el modelo LogisticRegression...")
    model_seasonal_vaccine = entrenar_y_evaluar_modelo_con_search(
        LogisticRegression(),
        param_logistic_regression,
        X,
        y_seasonal_vaccine,
        selected_features_names_seasonal_vaccine,
        'random'
    )

## RandomForestClassifier

In [ ]:
param_rf_classifier = {"n_estimators": [10, 50, 100],
              "max_depth": [3, 5, 10, None],
              "max_features": [1, 3, 5, 10],
              "min_samples_split": [2, 5, 10],
              "bootstrap": [True, False]}

### h1n1_vaccine

In [ ]:
if use_grid_search and use_random_forest_h1n1_vaccine:
    print("Vamos a realizar búsqueda de hiperparámetros para h1n1_vaccine con GridSearchCV y el modelo RandomForestClassifier...")
    model_h1n1_vaccine = entrenar_y_evaluar_modelo_con_search(
        RandomForestClassifier(random_state=42),
        param_rf_classifier,
        X,
        y_h1n1_vaccine,
        selected_features_names_h1n1_vaccine,
        'grid'
    )

In [ ]:
if use_random_search and use_random_forest_h1n1_vaccine:
    print("Vamos a realizar búsqueda de hiperparámetros para h1n1_vaccine con RandomSearchCV y el modelo RandomForestClassifier...")
    model_h1n1_vaccine = entrenar_y_evaluar_modelo_con_search(
        RandomForestClassifier(random_state=42),
        param_rf_classifier,
        X,
        y_h1n1_vaccine,
        selected_features_names_h1n1_vaccine,
        'random'
    )

### seasonal_vaccine

In [ ]:
if use_grid_search and use_random_forest_seasonal_vaccine:
    print("Vamos a realizar búsqueda de hiperparámetros para seasonal_vaccine con GridSearchCV y el modelo RandomForestClassifier...")
    model_seasonal_vaccine = entrenar_y_evaluar_modelo_con_search(
        RandomForestClassifier(random_state=42),
        param_rf_classifier,
        X,
        y_seasonal_vaccine,
        selected_features_names_seasonal_vaccine,
        'grid'
    )

In [ ]:
if use_random_search and use_random_forest_seasonal_vaccine:    
    print("Vamos a realizar búsqueda de hiperparámetros para seasonal_vaccine con RandomSearchCV y el modelo RandomForestClassifier...")
    model_seasonal_vaccine = entrenar_y_evaluar_modelo_con_search(
        RandomForestClassifier(random_state=42),
        param_rf_classifier,
        X,
        y_seasonal_vaccine,
        selected_features_names_seasonal_vaccine,
        'random'
    )

## AdaBoostClassifier

In [ ]:
param_adaboost_classifier = {
    'n_estimators': [50, 100, 200, 500], 
    'learning_rate': [0.001, 0.01, 0.1, 1, 10],  
    'algorithm': ['SAMME', 'SAMME.R'],  
    'random_state': [None, 0, 42, 100],  
    'estimator': [None, DecisionTreeClassifier(max_depth=1), DecisionTreeClassifier(max_depth=3)],  
}

### h1n1_vaccine

In [ ]:
if use_grid_search and use_ada_boost_classifier_h1n1_vaccine:
    print("Vamos a realizar búsqueda de hiperparámetros para h1n1_vaccine con GridSearchCV y el modelo AdaBoostClassifier...")
    model_h1n1_vaccine = entrenar_y_evaluar_modelo_con_search(
        AdaBoostClassifier(),
        param_adaboost_classifier,
        X,
        y_h1n1_vaccine,
        selected_features_names_h1n1_vaccine,
        'grid'
    )

In [ ]:
if use_random_search and use_ada_boost_classifier_h1n1_vaccine:
    print("Vamos a realizar búsqueda de hiperparámetros para h1n1_vaccine con RandomSearchCV y el modelo AdaBoostClassifier...")
    model_h1n1_vaccine = entrenar_y_evaluar_modelo_con_search(
        AdaBoostClassifier(),
        param_adaboost_classifier,
        X,
        y_h1n1_vaccine,
        selected_features_names_h1n1_vaccine,
        'random'
    )

### seasonal_vaccine

In [ ]:
if use_grid_search and use_ada_boost_classifier_seasonal_vaccine:
    print("Vamos a realizar búsqueda de hiperparámetros para seasonal_vaccine con GridSearchCV y el modelo AdaBoostClassifier...")
    model_seasonal_vaccine = entrenar_y_evaluar_modelo_con_search(
        AdaBoostClassifier(),
        param_adaboost_classifier,
        X,
        y_seasonal_vaccine,
        selected_features_names_seasonal_vaccine,
        'grid'
    )

In [ ]:
if use_random_search and use_ada_boost_classifier_seasonal_vaccine: 
    print("Vamos a realizar búsqueda de hiperparámetros para seasonal_vaccine con RandomSearchCV y el modelo AdaBoostClassifier...")
    model_seasonal_vaccine = entrenar_y_evaluar_modelo_con_search(
        AdaBoostClassifier(),
        param_adaboost_classifier,
        X,
        y_seasonal_vaccine,
        selected_features_names_seasonal_vaccine,
        'random'
    )

## XGBClassifier

In [ ]:
param_xgb_classifier = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5, 6],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.5, 0.6, 0.7],
    'gamma': [0, 0.1, 0.2, 0.3],
    'min_child_weight': [1, 3, 5]
}

### h1n1_vaccine

In [ ]:
if use_grid_search and use_xgb_classifier_h1n1_vaccine:
    print("Vamos a realizar búsqueda de hiperparámetros para h1n1_vaccine con GridSearchCV y el modelo XGBClassifier...")
    model_h1n1_vaccine = entrenar_y_evaluar_modelo_con_search(
        XGBClassifier(),
        param_xgb_classifier,
        X,
        y_h1n1_vaccine,
        selected_features_names_h1n1_vaccine,
        'grid'
    )

In [ ]:
if use_random_search and use_xgb_classifier_h1n1_vaccine:    
    print("Vamos a realizar búsqueda de hiperparámetros para h1n1_vaccine con RandomSearchCV y el modelo XGBClassifier...")
    model_h1n1_vaccine = entrenar_y_evaluar_modelo_con_search(
        XGBClassifier(),
        param_xgb_classifier,
        X,
        y_h1n1_vaccine,
        selected_features_names_h1n1_vaccine,
        'random'
    )

### seasonal_vaccine

In [ ]:
if use_grid_search and use_xgb_classifier_seasonal_vaccine:
    print("Vamos a realizar búsqueda de hiperparámetros para seasonal_vaccine con GridSearchCV y el modelo XGBClassifier...")
    model_seasonal_vaccine = entrenar_y_evaluar_modelo_con_search(
        XGBClassifier(),
        param_xgb_classifier,
        X,
        y_seasonal_vaccine,
        selected_features_names_seasonal_vaccine,
        'grid'
    )

In [ ]:
if use_random_search and use_xgb_classifier_seasonal_vaccine:      
    print("Vamos a realizar búsqueda de hiperparámetros para seasonal_vaccine con RandomSearchCV y el modelo XGBClassifier...")
    model_seasonal_vaccine = entrenar_y_evaluar_modelo_con_search(
        XGBClassifier(),
        param_xgb_classifier,
        X,
        y_seasonal_vaccine,
        selected_features_names_seasonal_vaccine,
        'random'
    )

## Red neuronal

In [ ]:
def entrenar_red_neuronal(X, y, top_features):
    X_train, X_test, y_train, y_test = split_x_y(X, y, top_features)

    model = Sequential([
        Dense(256, activation='relu', input_shape=(10,), kernel_regularizer=l2(0.01)),  # Capa densa con regularización L2
        BatchNormalization(),  # Normalización de lotes
        Dense(128, activation='relu', kernel_regularizer=l2(0.01)),  # Capa densa con regularización L2
        BatchNormalization(),  # Normalización de lotes
        Dense(64, activation='relu', kernel_regularizer=l2(0.01)),   # Capa densa con regularización L2
        Dense(1, activation='sigmoid')  # Capa de salida con 1 neurona y función de activación sigmoide para clasificación binaria
    ])

    # Compilar el modelo
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'AUC'])

    # Entrenar el modelo
    model.fit(X_train, y_train, epochs=epochs_red_neuronal, batch_size=32, validation_split=0.2)

    probabilidades = model.predict(X_test)

    roc_auc = roc_auc_score(y_test, probabilidades, average='macro')

    print("Área bajo la curva ROC (AUC):", roc_auc)
    
    return model

### h1n1_vaccine

In [ ]:
if use_red_neuronal_h1n1_vaccine:
    model_h1n1_vaccine = entrenar_red_neuronal(X, y_h1n1_vaccine, selected_features_names_h1n1_vaccine)

### seasonal_vaccine

In [ ]:
if use_red_neuronal_seasonal_vaccine:
    model_seasonal_vaccine = entrenar_red_neuronal(X, y_seasonal_vaccine, selected_features_names_seasonal_vaccine)

# Realizamos la predicción con el dataset de test

Codificamos las columnas con el mismo encoder que se ha hecho en el dataset de entrenamiento

In [ ]:
for col, le in encoders.items():
    if col in test.columns:
        test[col] = le.transform(test[col])

Identificar columnas con valores nulos

In [ ]:
columnas_con_nulos = test.columns[test.isnull().any()].tolist()

Rellenar las columnas con valores nulos con la moda de cada columna

In [ ]:
if use_moda_test:
    moda = test[columna].mode()[0]
    test[columna].fillna(moda, inplace=True)
    print(f'{columna} - {moda}')

Rellenamos los valores nulos con KNNImputer

In [ ]:
if use_knn_imputer_test:
    imputer_test = KNNImputer(n_neighbors=5)
    imputed_data_test = imputer.fit_transform(test)
    imputed_data_test = np.round(imputed_data_test).astype(int)
    test = pd.DataFrame(imputed_data_test, columns=test.columns, index=test.index)

Realizamos una copia de los datos de test para mantener los id de los encuestados

In [ ]:
test_copy = test.copy()

test = test.drop('respondent_id', axis=1)

Escalamos los datos con el mismo scaler que se ha utilizado en los datos de entrenamiento

In [ ]:
if not not_use_scaler:
    test_scaled = scaler.transform(test)

    test_df = pd.DataFrame(test_scaled, columns=test.columns)

test_df = test

Realizamos la predicción para h1n1_vaccine

In [ ]:
test_selected_features_names_h1n1_vaccine = test_df[selected_features_names_h1n1_vaccine]

probabilities_test_h1n1_vaccine = model_h1n1_vaccine.predict_proba(test_selected_features_names_h1n1_vaccine)[:, 1]

probabilities_test_h1n1_vaccine_truncated = [round(prob, 1) for prob in probabilities_test_h1n1_vaccine]

print("Probabilidades predichas para 'h1n1_vaccine' en el conjunto de prueba:")
print(probabilities_test_h1n1_vaccine_truncated)

Realizamos la predicción para seasonal_vaccine

In [ ]:
test_selected_features_names_seasonal_vaccine = test_df[selected_features_names_seasonal_vaccine]

probabilities_test_seasonal_vaccine = model_seasonal_vaccine.predict_proba(test_selected_features_names_seasonal_vaccine)[:, 1] 

probabilities_test_seasonal_vaccine_truncated = [round(prob, 1) for prob in probabilities_test_seasonal_vaccine]

print("Probabilidades predichas para 'seasonal_vaccine' en el conjunto de prueba:")
print(probabilities_test_seasonal_vaccine_truncated)

Mezclamos la columna del encuestado con las probabilidades predichas de h1n1_vaccine y seasonal_vaccine en un mismo DataFrame

In [ ]:
respondent_ids = test_copy['respondent_id']

submission_df = pd.DataFrame({
    'respondent_id': respondent_ids,
    'h1n1_vaccine': probabilities_test_h1n1_vaccine,  # Probabilidades predichas para la primera variable objetivo (h1n1_vaccine)
    'seasonal_vaccine': probabilities_test_seasonal_vaccine  # Probabilidades predichas para la segunda variable objetivo (seasonal_vaccine)
})

Generamos el archivo CSV para entregar en la competición añadiendo una marca de tiempo para identificarlos fácilmente

In [ ]:
now = datetime.now()
date = now.strftime("%Y-%m-%d")
time = now.strftime("%H-%M")

nombre_archivo = f'submissions/{date}/{time}/submission_{time}.csv'

os.makedirs(os.path.dirname(nombre_archivo), exist_ok=True)

submission_df.to_csv(nombre_archivo, index=False)

# Guardamos los modelos entrenados

In [ ]:
location_h1n1_vaccine = f'submissions/{date}/{time}/h1n1_vaccine_model.pkl'

location_seasonal_vaccine = f'submissions/{date}/{time}/seasonal_vaccine_model.pkl'

In [ ]:
with open(location_h1n1_vaccine, 'wb') as f:
    pickle.dump(model_h1n1_vaccine, f)
    
with open(location_seasonal_vaccine, 'wb') as f:
    pickle.dump(model_seasonal_vaccine, f)

# Tiempo total de ejecución

In [ ]:
end_time = datetime.now()
execution_time = end_time - start_time
print(f"Tiempo total de ejecución: {execution_time}")